In [39]:
import os
import re
import pandas as pd
import numpy as np
import plotly.express as px
import plotly.graph_objects as go
import ipywidgets as widgets
from IPython.display import display
from scipy.interpolate import griddata

In [40]:
usability_df = pd.read_csv('../data/adults/results/usability/usability_anonymized.csv')
anonymity_df = pd.read_csv('../data/adults/results/anonymization/anonymization_results.csv')

In [41]:
# Vamos a ordenar los 'QIDs' algabéticamente
usability_df['dataset'] = usability_df['dataset'].apply(lambda x: ', '.join(sorted(x.split(', '))))
anonymity_df['QIDs'] = anonymity_df['QIDs'].apply(lambda x: ', '.join(sorted(x.split(', '))))

In [42]:
# Hacemos merge de los dos DataFrames en base a los 'dataset' y 'QIDs' dejando los que coinciden
merged_df = pd.merge(usability_df, anonymity_df, left_on='dataset', right_on='QIDs', how='inner')

In [43]:
# Ahora añadimos el número de QIDs como una nueva columna
merged_df['num_QIDs'] = merged_df['QIDs'].apply(lambda x: len(x.split(', ')))

In [44]:
merged_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 11279 entries, 0 to 11278
Data columns (total 14 columns):
 #   Column                     Non-Null Count  Dtype  
---  ------                     --------------  -----  
 0   dataset                    11279 non-null  object 
 1   accuracy                   11279 non-null  float64
 2   precision                  11279 non-null  float64
 3   recall                     11279 non-null  float64
 4   f1_score                   11279 non-null  float64
 5   path                       11279 non-null  object 
 6   risk_distance              11279 non-null  float64
 7   QIDs                       11279 non-null  object 
 8   Generalization Levels      11279 non-null  object 
 9   estimated_journalist_risk  11279 non-null  object 
 10  estimated_prosecutor_risk  11279 non-null  object 
 11  estimated_marketer_risk    11279 non-null  object 
 12  sample_uniques             11279 non-null  object 
 13  num_QIDs                   11279 non-null  int

In [45]:
# Calcular medianas por número de QIDs
medians = merged_df.groupby('num_QIDs')[['accuracy', 'risk_distance']].median().reset_index()

x = medians['num_QIDs']
y_acc = medians['accuracy']
y_risk = medians['risk_distance']
y_ratio = y_risk / y_acc


fig = go.Figure()
fig.add_trace(go.Scatter(
    x=x, 
    y=y_acc, 
    mode='lines+markers', 
    name='Accuracy', 
    line=dict(color='blue', width=3),
    marker=dict(size=10, symbol='circle')

))
fig.add_trace(go.Scatter(
    x=x, 
    y=y_risk, 
    mode='lines+markers', 
    name='Risk Distance', 
    line=dict(color='red', width=3),
    marker=dict(size=10, symbol='circle')

))
fig.update_layout(
    # title='Medianas: Accuracy y Risk Distance por Número de QIDs',
    xaxis_title='Número de QIDs',
    yaxis_title='Métrica',
    legend=dict(
        orientation='h',
        yanchor='bottom',
        y=-0.25,
        xanchor='center',
        x=0.5,
        font=dict(size=16),
        title=dict(font=dict(size=18))
    ),
    xaxis=dict(
        tickmode='linear', 
        dtick=1,
        title_font=dict(size=22),
        tickfont=dict(size=16),
        tickvals=sorted(merged_df['num_QIDs'].unique()),
        ticktext=[str(int(val)) for val in sorted(merged_df['num_QIDs'].unique())]
    ),
    yaxis=dict(
        title_font=dict(size=22),
        tickfont=dict(size=16),
        # tickvals=[1, 0.8, 0.6, 0.4, 0.2, 0],
        # ticktext=[str(val) for val in [1, 0.8, 0.6, 0.4, 0.2, 0]],
        type='log'
    ),
    margin=dict(l=0, r=0, t=0, b=0),
    height=400,
    width=800
)
fig.write_image("../Memoria/images/graphs/usability-anonymity-vs-num_qids.png")
fig.show()


In [46]:
hier_path = '../data/adults/hierarchies/'
def load_hierarchies(hier_path) -> dict:
    """
    Carga las jerarquías desde archivos CSV en el directorio especificado.
    Cada archivo debe tener un nombre que comience con el atributo seguido de '_jerarquia.csv'.
    Devuelve un diccionario donde la clave es el atributo y el valor es un DataFrame de pandas.
    """
    hierarchies = {}
    for f in os.listdir(hier_path):
        if f.endswith('.csv'):
            attr = f.split('_')[0]  # Asumimos que el nombre del archivo es 'atributo_jerarquia.csv'
            hier_df = pd.read_csv(os.path.join(hier_path, f), index_col=0)
            # Convertimos el DataFrame a un diccionario donde el índice del DataFrame es la clave y la fila es el valor
            hierarchies[attr] = hier_df.to_dict(orient='dict')
    return hierarchies

def get_hierarchy_dict(levels: str) -> dict:
    """
    Obtiene un diccionario con la jerarquía de niveles proveniente de una cadena de la forma:
    \n**Ejemplo de entrada **: *"{'level1': '1/3', 'level2': '2/3', 'level3': '0/3'}"*.
    \nDevuelve un diccionario con la fracción del valor de cada nivel, excluyendo aquellos con valor 0.
    \n**Ejemplo de salida**: *{'level1': 1/3 , 'level2': 2/3}*.
    Args:
        levels (str): Cadena que representa los niveles y sus valores en formato 'nivel: valor/total'.
    Returns:
        dict: Diccionario con los niveles y sus valores, excluyendo aquellos con valor 0.
    """
    levels = re.findall(r"'[\w-]+': '\d/\d'", levels)  
    hier_dict =  {k.strip("'"): (l.strip("'").split("/")[0],l.strip("'").split("/")[1]) for k, l in (x.split(': ') for x in levels)}
    return {k: v for k, v in hier_dict.items()}

In [47]:
# Vamos a calcular la pérdida de información para cada uno de los dataframes
hierarchies = load_hierarchies(hier_path)

# Para una cadena de generalización de QIDs "{'age': '1/3', 'education': '2/3', 'occupation': '1/3'}"
# 1/num_total_qids * sum(generalization_level) -> 1/14 * (1/3 + 2/3 + 1/3) = 1/14 * 4/3 = 4/42 = 0.095238


def calculate_information_loss(generalization_str: str) -> float:
    generalization_dict = get_hierarchy_dict(generalization_str)
    v = []
    for k, (l, r) in generalization_dict.items():
        if l == '0':
            v.append(0)
            continue
        elif l == r:
            v.append(1)
            continue
        # Obtenemos un ejemplo entre los valores del nivel l de la jerarquía
        elem = list(hierarchies[k][l].values())[0]
        pattern = re.compile(r'\d+-\d+')
        # Si es nivel de generalización numérico
        if pattern.fullmatch(elem):
            # Extraemos el valor numérico del nivel de generalización
            num_values = [int(x) for x in elem.split('-')]
            # Calculamos la fracción de generalización
            generalization_fraction = (num_values[1]-num_values[0] + 1) / len(hierarchies[k]['1'])
        # Si es nivel de generalización categórico
        else:
            generalization_fraction = 1 - len(set(hierarchies[k][l].values())) / len(set(hierarchies[k]['1'].keys()))
        v.append(generalization_fraction)
    # Calculamos la pérdida de información
    num_total_qids = len(hierarchies)
    return (1 / num_total_qids) * sum(v)

In [48]:
calculate_information_loss("{'age': '4/5', 'sex': '1/1', 'education': '2/3'}")  # Ejemplo de uso

0.16249999999999998

In [49]:
# Añadimos la columna de información de pérdida al DataFrame
merged_df['information_loss'] = merged_df['Generalization Levels'].apply(calculate_information_loss)

In [50]:
fig_box = go.Figure()
fig_box.add_trace(go.Scatter(
    x=medians['num_QIDs'],
    y=medians['information_loss'] if 'information_loss' in medians else merged_df.groupby('num_QIDs')['information_loss'].median().values,
    mode='lines+markers',
    name='Information Loss',
    line=dict(color='green', width=3),
    marker=dict(size=10, symbol='circle')
))
fig_box.add_trace(go.Scatter(
    x=medians['num_QIDs'],
    y=medians['accuracy'],
    mode='lines+markers',
    name='Accuracy',
    line=dict(color='blue', width=3),
    marker=dict(size=10, symbol='circle')
))
fig_box.update_layout(
    xaxis_title='Número de QIDs',
    yaxis_title='Métrica',
    legend=dict(
        orientation='h',
        yanchor='bottom',
        y=-0.25,
        xanchor='center',
        x=0.5,
        font=dict(size=16),
        title=dict(font=dict(size=18))
    ),
    xaxis=dict(
        tickmode='linear', 
        dtick=1,
        title_font=dict(size=22),
        tickfont=dict(size=16),
        tickvals=sorted(merged_df['num_QIDs'].unique()),
        ticktext=[str(int(val)) for val in sorted(merged_df['num_QIDs'].unique())]
    ),
    yaxis=dict(
        title_font=dict(size=22),
        tickfont=dict(size=16),
        tickvals=[1, 0.8, 0.6, 0.4, 0.2, 0],
        ticktext=[str(val) for val in [1, 0.8, 0.6, 0.4, 0.2, 0]]
        ),
    margin=dict(l=0, r=0, t=0, b=0),
    height=400,
    width=800
)
fig_box.write_image("../Memoria/images/graphs/accuracy-and-info_loss-vs-num_qids.png")
fig_box.show()


In [51]:
import plotly.express as px

# Muestra solo una muestra aleatoria del 10% de los datos para reducir la cantidad de puntos
sampled_df = merged_df.sample(frac=0.1, random_state=42)

fig_acc_info = px.scatter(
    sampled_df,
    x='information_loss',
    y='accuracy',
    trendline='ols',
    labels={'information_loss': 'Pérdida de Información', 'accuracy': 'Accuracy'},
    # title='Relación entre Accuracy y Pérdida de Información'
)
fig_acc_info.update_traces(marker=dict(size=6, opacity=0.5))
fig_acc_info.show()

In [52]:
from IPython.display import clear_output

# Veamos cuales son los QIDs que más aparecen en el percentil <= 10 de distancia de riesgo
q_values = [0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9]

top_risk_dict = {q: merged_df[merged_df['risk_distance'] <= merged_df['risk_distance'].quantile(q)] for q in q_values}
top_acc_dict = {q: merged_df[merged_df['accuracy'] > merged_df['accuracy'].quantile(1-q)] for q in q_values}

# Aplicamos map a los valores del diccionario para obtener las listas de QIDs
top_risk_qids = {q: df['QIDs'].apply(lambda x: x.split(', ')).tolist() for q, df in top_risk_dict.items()}
top_acc_qids = {q: df['QIDs'].apply(lambda x: x.split(', ')).tolist() for q, df in top_acc_dict.items()}


fig_1_dict = {}
# Precalculamos las figuras que muestran con un gráfico de barras los QIDs más frecuentes para un percentil dado
for q in q_values:
    # Nos quedamos con las sublistas de QIDs que están en ambos diccionarios como valores
    qids_list = [sublist for sublist in top_risk_qids[q] if sublist in top_acc_qids[q] and 'sex' in sublist and 'age' in sublist]
    # Aplanamos la lista de listas en una sola lista
    qids_flat = [qid for sublist in qids_list for qid in sublist]
    # Contamos las ocurrencias de cada QID
    qid_counts = pd.Series(qids_flat).value_counts().reset_index()
    qid_counts.columns = ['QID', 'Count']
    
    
    fig_risk = px.bar(
        # Quitamos sex y age de los QIDs para enfocarnos en los otros atributos
        qid_counts[~qid_counts['QID'].isin(['age', 'sex', 'fnlwgt'])],
        x='QID',
        y='Count',
        # title=f'Top QIDs más seleccionados que verifican Acc > {(1-q)*100}% y Risk <= {q*100}%',
        labels={'QID': 'QIDs', 'Count': 'Frecuencia'}
    )
    fig_risk.update_xaxes(
        title_font=dict(size=20),
        tickfont=dict(size=18),
        tickangle=30
    )
    fig_risk.update_layout(margin=dict(l=0, r=0, t=0, b=0))
    fig_1_dict[q] = fig_risk

# Creamos un widget de selección para elegir el percentil
percentil_selector = widgets.IntSlider(
    min=10,
    max=90,
    step=10,
    value=10,
    description='Percentil:',
    readout=True,
    readout_format='d'
)
output = widgets.Output()

def update_plot(change):
    with output:
        clear_output(wait=True)
        selected_percentil = change['new'] / 100
        fig_1_dict[selected_percentil].show()

percentil_selector.observe(update_plot, names='value')
display(percentil_selector, output)
# Mostramos el gráfico del percentil por defecto (10%)
with output:
    fig_1_dict[0.1].show()


IntSlider(value=10, description='Percentil:', max=90, min=10, step=10)

Output()

In [53]:
from IPython.display import clear_output

# Veamos cuales son los QIDs que más aparecen en el percentil <= 10 de distancia de riesgo
q_values = [0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9]

top_risk_dict = {q: merged_df[merged_df['risk_distance'] > merged_df['risk_distance'].quantile(1-q)] for q in q_values}
top_acc_dict = {q: merged_df[merged_df['accuracy'] >= merged_df['accuracy'].quantile(q)] for q in q_values}

# Aplicamos map a los valores del diccionario para obtener las listas de QIDs
top_risk_qids = {q: df['QIDs'].apply(lambda x: x.split(', ')).tolist() for q, df in top_risk_dict.items()}
top_acc_qids = {q: df['QIDs'].apply(lambda x: x.split(', ')).tolist() for q, df in top_acc_dict.items()}


fig_2_dict = {}
# Precalculamos las figuras que muestran con un gráfico de barras los QIDs más frecuentes para un percentil dado
for q in q_values:
    # Nos quedamos con las sublistas de QIDs que están en ambos diccionarios como valores
    qids_list = [sublist for sublist in top_risk_qids[q] if sublist in top_acc_qids[q] and 'sex' in sublist and 'age' in sublist]
    # Aplanamos la lista de listas en una sola lista
    qids_flat = [qid for sublist in qids_list for qid in sublist]
    # Contamos las ocurrencias de cada QID
    qid_counts = pd.Series(qids_flat).value_counts().reset_index()
    qid_counts.columns = ['QID', 'Count']
    
    
    fig_risk = px.bar(
        # Quitamos sex y age de los QIDs para enfocarnos en los otros atributos
        qid_counts[~qid_counts['QID'].isin(['age', 'sex', 'fnlwgt'])],
        x='QID',
        y='Count',
        # title=f'Top QIDs más seleccionados que verifican Acc <= {int(q*100)}% y Risk > {int((1-q)*100)}%',
        labels={'QID': 'QIDs', 'Count': 'Frecuencia'}
        )
    fig_risk.update_xaxes(
        title_font=dict(size=20),
        tickfont=dict(size=18),
        tickangle=30
    )
    fig_risk.update_xaxes(
        title_font=dict(size=20),
        tickfont=dict(size=18),
    )
    fig_risk.update_layout(margin=dict(l=0, r=0, t=0, b=0))
    fig_2_dict[q] = fig_risk

# Creamos un widget de selección para elegir el percentil
percentil_selector = widgets.IntSlider(
    min=10,
    max=90,
    step=10,
    value=10,
    description='Percentil:',
    readout=True,
    readout_format='d'
)
output = widgets.Output()

def update_plot(change):
    with output:
        clear_output(wait=True)
        selected_percentil = change['new'] / 100
        fig_2_dict[selected_percentil].show()

percentil_selector.observe(update_plot, names='value')
display(percentil_selector, output)
# Mostramos el gráfico del percentil por defecto (10%)
with output:
    fig_2_dict[0.1].show()


IntSlider(value=10, description='Percentil:', max=90, min=10, step=10)

Output()